In [1]:
sniffingDir = "data/train/Sniffing"
backgroundDir = "data/train/Background"
testSniffingDir = "data/test/Sniffing"
testBackgroundDir = "data/test/Background"

validation_fraction = 0.2

correct_class_imbalance = False

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
import tensorflow_io as tfio
import os
import pathlib
from dvc.api import DVCFileSystem

#import numpy as np
import tensorflow.experimental.numpy as tnp
from tensorflow.python.ops import gen_audio_ops as audio_ops

#  enable NumPy behavior for TensorFlow:
tnp.experimental_enable_numpy_behavior()

In [13]:
def generate_spectrogram(file_path, label):
    # does not scale as sniffing should be the same independent of background level
    audio_tensor = tfio.audio.AudioIOTensor(file_path, dtype=tf.int16)
    audio = tf.cast(audio_tensor[:], tf.float32)
    spectrogram = audio_ops.audio_spectrogram(audio,
                                              window_size=320,
                                              stride=160,
                                              magnitude_squared=True)
    spectrogram = tf.nn.pool(
        input=tf.expand_dims(spectrogram, -1),
        window_shape=[1, 6],
        strides=[1, 6],
        pooling_type='AVG',
        padding='SAME')
    spectrogram = tf.squeeze(spectrogram, axis=0)
    # Not sure whether the log is a good idea...
    spectrogram = tnp.log10(spectrogram + 1e-6)
    return spectrogram, label

def prepare_data(dir, value):
    filePath = os.path.join(dir, "*.wav")
    files = tf.data.Dataset.list_files(filePath)
    file_count = len(files)
    #values = tf.zeros(len(files)) if value == 0 else tf.ones(len(files))
    if value == 0:
        values = tf.concat([tf.zeros(shape=[file_count, 1]), tf.ones(shape=[file_count, 1])], axis=1)
    else:
        values = tf.concat([tf.ones(shape=[file_count, 1]), tf.zeros(shape=[file_count, 1])], axis=1)

    data = tf.data.Dataset.zip((files, tf.data.Dataset.from_tensor_slices(values)))
    spectrogramData = data.map(generate_spectrogram)
    return spectrogramData

def generate_binary_dataset(trueDataDir, falseDataDir):
    trueData = prepare_data(trueDataDir, 1)
    true_number = trueData.cardinality().numpy()
    print(f"Sniffing Datasets: {true_number}")
    falseData = prepare_data(falseDataDir, 0)
    false_number = falseData.cardinality().numpy()
    print(f"Background Datasets: {false_number}")
    if correct_class_imbalance:
        falseData = falseData.shuffle(false_number).take(true_number)
        false_number = falseData.cardinality().numpy()
        print(f"Corrected Background Datasets: {false_number}")
    combinedData = trueData.concatenate(falseData)
    combinedData = combinedData.cache()
    return combinedData.shuffle(buffer_size=combinedData.cardinality().numpy())

def split_validation(allData, fraction):
    numberDataSets = allData.cardinality().numpy()
    validationCount = round(numberDataSets * fraction)
    trainSet = allData.take(numberDataSets - validationCount)
    validationSet = allData.skip(numberDataSets - validationCount).take(validationCount)
    return trainSet, validationSet

def batch_prefetch(dataSet):
    dataSet = dataSet.batch(16)
    dataSet = dataSet.prefetch(8)
    return dataSet

def calc_accuracy(predictions, true_values):
    accuracy = sum(map(lambda x, y: x == y == 1, true_values, predictions))/sum(true_values)
    return accuracy

def representative_data_gen():
    repr_samples, repr_labels = train.as_numpy_iterator().next()
    yield [repr_samples]

In [14]:
data = generate_binary_dataset(trueDataDir=sniffingDir, falseDataDir=backgroundDir)
train, validation = split_validation(data, validation_fraction)
train = batch_prefetch(train)
validation = batch_prefetch(validation)
samples, labels = train.as_numpy_iterator().next()
inputShape = samples.shape[1:]
print(f"inputShape {inputShape}")

Sniffing Datasets: 251
Background Datasets: 772
inputShape (99, 43, 1)


In [15]:
model = Sequential([
    Conv2D(4, 3,
           padding='same',
           activation='relu',
           name='conv_layer1',
           input_shape=inputShape),
    MaxPooling2D(name='max_pooling1', pool_size=(2,2)),
    Conv2D(4, 3,
           padding='same',
           activation='relu',
           name='conv_layer2'),
    MaxPooling2D(name='max_pooling2', pool_size=(2,2)),
    Flatten(),
    Dropout(0.2),
    Dense(
        40,
        activation='relu',
        name='hidden_layer1'
    ),
    Dense(
        2, # TODO test 2 output nodes
        activation='softmax',
        name='output'
    )
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_layer1 (Conv2D)        (None, 99, 43, 4)         40        
                                                                 
 max_pooling1 (MaxPooling2D)  (None, 49, 21, 4)        0         
                                                                 
 conv_layer2 (Conv2D)        (None, 49, 21, 4)         148       
                                                                 
 max_pooling2 (MaxPooling2D)  (None, 24, 10, 4)        0         
                                                                 
 flatten_3 (Flatten)         (None, 960)               0         
                                                                 
 dropout_3 (Dropout)         (None, 960)               0         
                                                                 
 hidden_layer1 (Dense)       (None, 40)               

In [19]:
hist = model.fit(train, epochs=100, validation_data=validation)

Epoch 1/100
52/52 [==============================] - 1s 14ms/step - loss: 0.2912 - recall_2: 0.8826 - precision_2: 0.8826 - val_loss: 0.2479 - val_recall_2: 0.9073 - val_precision_2: 0.9073
Epoch 2/100
52/52 [==============================] - 1s 13ms/step - loss: 0.2725 - recall_2: 0.8863 - precision_2: 0.8863 - val_loss: 0.2598 - val_recall_2: 0.9073 - val_precision_2: 0.9073
Epoch 3/100
52/52 [==============================] - 1s 13ms/step - loss: 0.3124 - recall_2: 0.8692 - precision_2: 0.8692 - val_loss: 0.1999 - val_recall_2: 0.9268 - val_precision_2: 0.9268
Epoch 4/100
52/52 [==============================] - 1s 13ms/step - loss: 0.2621 - recall_2: 0.8985 - precision_2: 0.8985 - val_loss: 0.2049 - val_recall_2: 0.9317 - val_precision_2: 0.9317
Epoch 5/100
52/52 [==============================] - 1s 13ms/step - loss: 0.2589 - recall_2: 0.8961 - precision_2: 0.8961 - val_loss: 0.1870 - val_recall_2: 0.9317 - val_precision_2: 0.9317
Epoch 6/100
52/52 [==============================]

In [20]:
testData = generate_binary_dataset(trueDataDir=testSniffingDir, falseDataDir=testBackgroundDir)
testData = batch_prefetch(testData)

model.evaluate(testData)

Sniffing Datasets: 64
Background Datasets: 192
16/16 [==============================] - 1s 6ms/step - loss: 0.7549 - recall_2: 0.7969 - precision_2: 0.7969


[0.7548670768737793, 0.796875, 0.796875]